In [0]:
import tensorflow as tf
import tensorflow.keras as keras
print(tf.__version__)
print(keras.__version__)

1.14.0
2.2.4-tf


## GoogleDriveをマウントする。

In [2]:
import os
if os.name == 'nt':
    print('OS is Windows: PASS mount google drive')
    g_dir_work = '../colab/'
else:
    from google.colab import drive
    drive.mount('/content/drive')
    g_dir_work = '/content/drive/My Drive/colab/'

# check mount point
print('\n<< Display work dir >>')
for file in os.listdir(g_dir_work):
    print( 'file/dir : ', file)

OS is Windows: PASS mount google drive

<< Display work dir >>
file/dir :  .ipynb_checkpoints
file/dir :  data
file/dir :  mnist_cams
file/dir :  mnist_pix2pix
file/dir :  models
file/dir :  mylib
file/dir :  template.ipynb
file/dir :  test.txt
file/dir :  tmp


## local Libraryパスを通す。

In [3]:
import sys
if os.name == 'nt':# windows
    lib_path='../colab/mylib'
else:
    print(os.getcwd())
    lib_path='/content/drive/My Drive/colab/mylib/'
sys.path.append(lib_path)

In [4]:
# check lib path
import myfunc
myfunc.test()

myfunc.test


以上、テンプレート

---

### データの前処理

In [0]:
import datasets

data = datasets.Dataset_mnist()
data.load()
data.x_train = data.x_train.reshape(data.x_train.shape[0], data.x_train.shape[1], data.x_train.shape[2], 1) / 255.
data.x_val   = data.x_val.reshape(data.x_val.shape[0], data.x_val.shape[1], data.x_val.shape[2], 1)   / 255.

import tensorflow.keras as keras
import numpy as np
num_classes = 10
data.y_train = keras.utils.to_categorical(data.y_train.astype(np.int8), num_classes)
data.y_val   = keras.utils.to_categorical(data.y_val.astype(np.int8), num_classes)

Using TensorFlow backend.


11493376/11490434 [==============================] - 16s 1us/step


In [0]:
data.dbg_check()

(60000, 28, 28, 1)
(60000, 10)
(10000, 28, 28, 1)
(10000, 10)


### 訓練用モデル構築

In [5]:
import models
model = models.Model_mnist_gradcam(b_training=True)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
imgs (InputLayer)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_______________________________________

NameError: name 'keras' is not defined

### モデルの最適化

In [0]:
batch_size = 1000
epochs = 10

In [0]:
print(data.y_train.shape)
print(data.x_train.shape)
print(data.y_val.shape)
print(data.x_val.shape)

(60000, 10)
(60000, 28, 28, 1)
(10000, 10)
(10000, 28, 28, 1)


In [0]:
history = model.fit(data.x_train, data.y_train,  # 画像とラベルデータ
                    batch_size=batch_size,
                    epochs=epochs,     # エポック数の指定
                    verbose=1,         # ログ出力の指定. 0だとログが出ない
                    validation_data=(data.x_val, data.y_val))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 112us/sample - loss: 0.9892 - acc: 0.6803 - val_loss: 0.1861 - val_acc: 0.9491
Epoch 2/10
60000/60000 [==============================] - 3s 54us/sample - loss: 0.2790 - acc: 0.9178 - val_loss: 0.0966 - val_acc: 0.9718
Epoch 3/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.1663 - acc: 0.9517 - val_loss: 0.0666 - val_acc: 0.9793
Epoch 4/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.1250 - acc: 0.9657 - val_loss: 0.0576 - val_acc: 0.9828
Epoch 5/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.1024 - acc: 0.9714 - val_loss: 0.0493 - val_acc: 0.9838
Epoch 6/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.0840 - acc: 0.9767 - val_loss: 0.0364 - val_acc: 0.9883
Epoch 7/10
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0758 - acc: 0.9793 - val_loss: 0

In [0]:
score = model.evaluate(data.x_val, data.y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.031874676238253595
Test accuracy: 0.989


### モデルの保存

In [0]:
import datetime
dt_now = datetime.datetime.now()

save_dir = '{0}/models/mnist_gradcam/{1}/'.format(g_dir_work, dt_now.strftime('%Y%m%d_%H%M%S') )
print(save_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model.save( save_dir + '/model.h5' )

/content/drive/My Drive/colab//models/mnist_gradcam/20190814_095018/


In [0]:
del model

---

### 推定モデルの読み込み

In [0]:
model_pred = models.Model_mnist_gradcam(b_training=False, model_path=save_dir + '/model.h5')

@@@@@@@@@ for prediction
Model: "model_mnist_gradcam_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
imgs (InputLayer)               [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 28, 28, 32)   320         imgs[0][0]                       
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 14, 14, 32)   0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 12, 12, 64)   18496       max_pooling2d_3[0][0]            
_____________________________________________________

In [0]:
import tensorflow.keras.backend as K

def get_cam(model, x, y):    
    result = model.predict([y.reshape((1, 10)), x.reshape((1, 28, 28, 1))])
    print('answer: ', K.eval(K.argmax(y)))
    print('prediction: ', K.eval(K.argmax(result[0])))

    conv_grad = result[1]
    conv_grad = conv_grad.reshape(conv_grad.shape[1:])
    conv_output = result[2]
    conv_output = conv_output.reshape(conv_output.shape[1:])
    input_grad = result[3]
    input_grad = input_grad.reshape(input_grad.shape[1:])
    gradRGB = gb_viz = input_grad

    from skimage.transform import resize
    import cv2

    # global average pooling
    weights = np.mean(conv_grad, axis = (0, 1))
    cam = np.zeros(conv_output.shape[0 : 2], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * conv_output[:, :, i]

    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)
    cam = resize(cam, (28,28), preserve_range=True)

    img = x.astype(float)
    #img -= np.min(img)
    #img /= img.max()

    cam_heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
    cam_heatmap = cv2.cvtColor(cam_heatmap, cv2.COLOR_BGR2RGB)


    cam = np.float32(cam.reshape((28, 28, 1))) * np.float32(img)
    cam = 255 * cam / np.max(cam)
    cam = np.uint8(cam)

    return img, cam, cam_heatmap

In [0]:
from PIL import Image
def save_img(img, file):
    img[ img <   0 ] = 0
    img[ img > 255 ] = 255
    if os.path.exists(os.path.dirname(file)) is False:
        os.makedirs( os.path.dirname(file) )
    Image.fromarray(img.astype(np.uint8)).save(file)

In [0]:
for i in range(20):
    x = data.x_val[i]
    y = data.y_val[i]

    img, cam, cam_heatmap = get_cam(model_pred, x, y)
    
    shape = (img.shape[0], img.shape[1])    
    overlay = np.zeros((img.shape[0], img.shape[1], 3))
    overlay[:,:,0] = img.reshape(shape)*128 + cam_heatmap[:,:,0]*0.5
    overlay[:,:,1] = img.reshape(shape)*128 + cam_heatmap[:,:,1]*0.5
    overlay[:,:,2] = img.reshape(shape)*128 + cam_heatmap[:,:,2]*0.5
    save_img( img.reshape(shape)*255, g_dir_work + 'mnist_cams/{0:03d}_original.png'.format(i) )
    save_img( cam.reshape(shape)*255, g_dir_work + 'mnist_cams/{0:03d}_cam.png'.format(i) )
    save_img( overlay, g_dir_work + 'mnist_cams/{0:03d}_overlaied.png'.format(i) )

answer:  7
prediction:  [7]
answer:  2
prediction:  [2]
answer:  1
prediction:  [1]
answer:  0
prediction:  [0]
answer:  4
prediction:  [4]
answer:  1
prediction:  [1]
answer:  4
prediction:  [4]
answer:  9
prediction:  [9]
answer:  5
prediction:  [5]
answer:  9
prediction:  [9]
answer:  0
prediction:  [0]
answer:  6
prediction:  [6]
answer:  9
prediction:  [9]
answer:  0
prediction:  [0]
answer:  1
prediction:  [1]
answer:  5
prediction:  [5]
answer:  9
prediction:  [9]
answer:  7
prediction:  [7]
answer:  3
prediction:  [3]
answer:  4
prediction:  [4]
